In [2]:
import os
import io
import PIL
from PIL import Image
import fitz
import pdfplumber
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from IPython.display import display
import PySimpleGUI as sg
import csv
import re
import docx
from docx.shared import RGBColor
from docx.shared import Cm, Pt  #加入可調整的 word 單位
from docx.oxml.ns import qn
from docx.enum.text import WD_ALIGN_PARAGRAPH #處理字串的置中

In [3]:
pdf_dir = input('Please enter PFT PDF folder name: ')
# data_dir = input('Please enter the folder name for saving data ')
# im_dir = input('Please input image folder: ')
print(os.listdir(pdf_dir)[1])
print(pdf_dir)


吳欣蘭_07011694_20210315145952.pdf
C:\Users\drcho\OneDrive\文件\Python Files\6MWT\Vitagraph PDFs


In [4]:
def pft_to_dict(folder):
    dict_p = {}
    for file_n in os.listdir(folder):
        file = pdfplumber.open(r"{}\{}".format(folder, file_n)) #讀取檔案內容
        dict_p[file_n] = [file.pages[0], file.pages[1]]
    return dict_p


def extract_basic_data(pdf_dict):
    columns_basic_data = ['Chart No.', 'Name', 'Birthday', "Age", 'Gender', 'BMI', 'Test-date']
    bdata_df = pd.DataFrame()
    for key in pdf_dict:
        table_0 = pdf_dict[key][0].extract_tables()
        basic_data = [[table_0[0][1][3], table_0[0][1][1], table_0[0][2][3], int(table_0[0][3][1]), table_0[0][4][1]
, float(table_0[0][5][1]), table_0[1][1][1].split(' ')[0]]]
        if bdata_df.empty:
            bdata_df = pd.DataFrame(basic_data, columns=columns_basic_data)
        else :
            bd_df = pd.DataFrame(basic_data, columns=columns_basic_data)
            bdata_df = bdata_df.append(bd_df, ignore_index=True)
    return bdata_df
            


In [5]:
p_dict = pft_to_dict(pdf_dir)
# p_dict

In [6]:
df_1 = extract_basic_data(p_dict)
df_1

,Chart No.,Name,Birthday,Age,Gender,BMI,Test-date
0,05034552,劉張美雲,05/05/1946,74,Female,24.3,03/05/2021
1,07011694,吳欣蘭,05/05/1986,34,Female,22.5,15/03/2021
2,18203194,姚凱仁,26/08/1982,38,Male,32.1,03/05/2021
3,16885750,廖淑瑛,31/10/1952,68,Female,25.9,13/05/2021
4,05740668,張和明,10/05/1940,80,Male,25.9,26/04/2021
5,02408329,強照文,13/04/1964,57,Male,30.5,03/05/2021
6,06022471,彭冠儒,05/10/1974,46,Male,16.2,15/03/2021
7,14523803,徐然基,27/04/1956,64,Male,21.1,26/04/2021
8,17931258,朱婉芳,03/08/1954,66,Female,24.0,06/05/2021
9,07801534,林宜穎,31/08/1943,77,Female,15.3,05/04/2021


In [7]:
def extract_pft_data(pdf_dict):
    pft_df = pd.DataFrame()
    for key in pdf_dict:
        table = pdf_dict[key][1].extract_table()
        columns_new = [item.replace('\n', '-') for item in table[1]]
        for lst in table[2:18]:
            for n in range(1, len(table[2])):
                try: 
                    lst[n]=float(lst[n].replace('*', ''))
                except ValueError:
                    lst[n]=lst[n].replace('*', '')
        temp_df = pd.DataFrame(table[2:18], columns=columns_new)
        temp_df_n = temp_df.set_index("Parameter")
        new_row = ['FVC (L)', 'FEV1 (L)', 'FEV1/FVC', 'PEF (L/min)', 'FEF25-75 (L/s)'
        , 'FIVC (L)', 'FIVC/FVC', 'FIF50/FEF50', 'TV (L)', 'IC (L)']
        temp_df_n = temp_df_n.loc[new_row, :]
        pre_best = list(temp_df_n['Pre-Best'])
        pre_pred_p = list(temp_df_n['% Pre-Pred'])
        post_best = list(temp_df_n['Post-Best'])
        columns_name = list(temp_df_n.index)
        column_pre = ["Pre-"+ name for name in columns_name]
        column_pred = [r"%Pred "+ name.split(' ')[0] for name in columns_name]
        column_post = ["Post-"+ name for name in columns_name]
        columns_new = column_pre + column_pred + column_post
        data_lst = pre_best+pre_pred_p+post_best
        data_lst_lst =[]
        for item in data_lst:
            data_lst_lst.append([item])
        data_dict = dict(zip(columns_new, data_lst_lst))
        temp_df_nn = pd.DataFrame.from_dict(data_dict)
        # temp_df_nn.insert(0, 'Chart No.', df_1['Chart No.'])
        # temp_df_nn['Test-date'] = df_1['Test-date']
        if pft_df.empty:
            pft_df = temp_df_nn
        else:
            pft_df = pft_df.append(temp_df_nn, ignore_index=True)
    return pft_df

In [8]:
df_2 = extract_pft_data(p_dict)
df_2.insert(0, 'Chart No.', df_1['Chart No.'])
df_2['Test-date'] = df_1['Test-date']
df_2

,Chart No.,Pre-FVC (L),Pre-FEV1 (L),Pre-FEV1/FVC,Pre-PEF (L/min),Pre-FEF25-75 (L/s),Pre-FIVC (L),Pre-FIVC/FVC,Pre-FIF50/FEF50,Pre-TV (L),...,Post-FEV1 (L),Post-FEV1/FVC,Post-PEF (L/min),Post-FEF25-75 (L/s),Post-FIVC (L),Post-FIVC/FVC,Post-FIF50/FEF50,Post-TV (L),Post-IC (L),Test-date
0,05034552,1.59,0.90,0.57,177.0,0.49,1.63,1.03,4.27,0.60,...,0.86,0.51,166.0,0.47,1.66,0.99,4.52,0.72,1.27,03/05/2021
1,07011694,2.84,2.68,0.94,351.0,3.97,2.85,1.00,0.74,0.50,...,2.59,0.96,369.0,3.99,2.61,0.97,0.68,1.30,1.80,15/03/2021
2,18203194,4.09,3.48,0.85,365.0,4.14,3.85,0.94,0.77,1.23,...,3.51,0.85,562.0,4.39,3.91,0.95,0.79,1.72,2.69,03/05/2021
3,16885750,1.21,0.63,0.52,110.0,0.28,1.06,0.88,2.55,0.57,...,0.64,0.53,98.0,0.28,1.13,0.93,4.50,0.60,0.98,13/05/2021
4,05740668,2.50,1.96,0.78,577.0,1.64,2.58,1.03,1.41,0.57,...,2.01,0.80,579.0,1.83,2.61,1.04,1.21,0.72,1.47,26/04/2021
5,02408329,3.30,2.53,0.77,532.0,2.01,3.27,0.99,1.72,0.80,...,2.61,0.76,555.0,2.02,3.37,0.98,1.32,1.71,2.33,03/05/2021
6,06022471,2.25,2.20,0.98,341.0,4.07,2.12,0.94,1.02,1.02,...,2.05,0.97,260.0,3.41,2.04,0.97,1.14,1.02,1.41,15/03/2021
7,14523803,3.45,2.63,0.76,329.0,2.14,3.31,0.96,1.11,1.53,...,2.67,0.79,383.0,2.27,3.24,0.96,1.36,1.69,1.89,26/04/2021
8,17931258,1.85,1.52,0.82,340.0,1.60,1.90,1.03,0.70,0.71,...,1.55,0.83,262.0,1.82,1.93,1.04,0.69,0.96,1.54,06/05/2021
9,07801534,1.03,0.75,0.73,189.0,0.43,0.99,0.96,0.50,0.41,...,0.73,0.72,207.0,0.45,1.00,0.98,0.55,0.39,0.70,05/04/2021


In [9]:
range(len(df_1))

range(0, 27)

In [10]:
data_dir = input('Please enter data saving folder name: ')
data_name = input('Please enter file name ')
df_1.to_excel(r'{}\{}_01.xlsx'.format(data_dir, data_name), index=False)
df_1.to_csv(r'{}\{}_01.csv'.format(data_dir, data_name), index=False)
df_2.to_excel(r'{}\{}_02.xlsx'.format(data_dir, data_name), index=False)
df_2.to_csv(r'{}\{}_02.csv'.format(data_dir, data_name), index=False)
